In [29]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [1]:
from OCC.Core.BRep import BRep_Tool
from OCC.Core.TopoDS import topods
from OCC.Core.gp import gp_Pnt
from OCC.Core.Geom import Geom_Curve
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_Transform
from OCC.Core.gp import gp_Circ, gp_Ax2, gp_Pnt, gp_Dir
from OCC.Extend.TopologyUtils import TopologyExplorer
from OCC.Core.BRepGProp import brepgprop_SurfaceProperties
from OCC.Core.BRepAdaptor import BRepAdaptor_Surface
from OCC.Core.GeomAbs import GeomAbs_Cylinder, GeomAbs_Circle

def extract_points_from_shape(shape):
    # Get the shape's edges and extract points along the edges
    points = []
    explorer = TopologyExplorer(shape)
    
    for edge in explorer.edges():
        # Use BRep_Tool to get the geometry of the edge
        curve, first, last = BRep_Tool.Curve(edge)  # Unpack the tuple to get the curve and parameter range
        
        # Get the first point on the curve
        pnt = curve.Value(first)  # Get the point at the starting parameter 'first'
        points.append(pnt)
        
    return points

def write_nc1_file(points, filename="part.nc1"):
    with open(filename, "w") as file:
        file.write("N10; Header information\n")
        file.write("N20; Begin shape data\n")
        
        # Write points as coordinates
        for idx, point in enumerate(points):
            x, y, z = point.X(), point.Y(), point.Z()
            file.write(f"X{x:.2f} Y{y:.2f} Z{z:.2f}; Point {idx+1}\n")
        
        file.write("N99; End of file\n")

In [2]:
from OCC.Core.GeomAbs import GeomAbs_Cylinder, GeomAbs_Circle
from OCC.Core.BRepAdaptor import BRepAdaptor_Surface, BRepAdaptor_Curve
from OCC.Core.TopAbs import TopAbs_FACE
from OCC.Extend.TopologyUtils import TopologyExplorer
from OCC.Core.Bnd import Bnd_Box
from OCC.Core.BRepBndLib import brepbndlib
from OCC.Core.gp import gp_Dir, gp_Vec, gp_XYZ
from OCC.Core.BRepLProp import BRepLProp_SLProps

import math

def is_axis_aligned_with_face(face, axis_dir, tolerance=0.2):
    # Get the surface of the face
    surf_adaptor = BRepAdaptor_Surface(face)
    
    # Mid-point of the surface (to check normal)
    u_min, u_max = surf_adaptor.FirstUParameter(), surf_adaptor.LastUParameter()
    v_min, v_max = surf_adaptor.FirstVParameter(), surf_adaptor.LastVParameter()
    u_mid, v_mid = (u_min + u_max) / 2, (v_min + v_max) / 2

    # Get the normal at the surface's mid-point
    props = BRepLProp_SLProps(surf_adaptor, u_mid, v_mid, 1, 1e-6)
    if not props.IsNormalDefined():
        return False
    
    normal = props.Normal()
    
    # Convert normal to gp_Vec (instead of gp_Dir)
    normal_vec = gp_Vec(normal.X(), normal.Y(), normal.Z())  # Ensure correct type
    
    # Convert axis direction to gp_Vec (make sure it's a vector)
    axis_vec = gp_Vec(axis_dir.X(), axis_dir.Y(), axis_dir.Z())  # Ensure correct type
    
    # Calculate the angle between the two vectors
    angle = normal_vec.Angle(axis_vec)
    
    # Check if the angle is within tolerance of being aligned (0 or pi radians)
    return abs(angle) < tolerance or abs(angle - math.pi) < tolerance


def detect_holes(solid, min_hole_diameter=1.0, max_hole_diameter=100.0, min_circular_arc=92.0):
    """
    Detects cylindrical holes in a solid.

    Parameters:
    - min_hole_diameter: Minimum diameter to be considered a hole.
    - max_hole_diameter: Maximum diameter.
    - min_circular_arc: Minimum angular sweep in degrees to qualify as a hole (e.g., 300° to exclude quarter circles).

    Returns:
    - List of hole dictionaries with center, direction, radius, diameter.
    """
    hole_data = []
    explorer = TopologyExplorer(solid)

    for face in explorer.faces():
        surf_adaptor = BRepAdaptor_Surface(face, True)
        surf_type = surf_adaptor.GetType()

        if surf_type != GeomAbs_Cylinder:
            continue

        cylinder = surf_adaptor.Cylinder()
        radius = cylinder.Radius()
        axis = cylinder.Axis()
        location = cylinder.Location()
        diameter = 2 * radius

        if not (min_hole_diameter <= diameter <= max_hole_diameter):
            continue

        # Analyze circular edges on this face
        face_explorer = TopologyExplorer(face)
        valid_circular_edge = False

        for edge in face_explorer.edges():
            curve_adaptor = BRepAdaptor_Curve(edge)
            curve_type = curve_adaptor.GetType()

            if curve_type != GeomAbs_Circle:
                continue

            u_start = curve_adaptor.FirstParameter()
            u_end = curve_adaptor.LastParameter()
            arc_angle_deg = abs(math.degrees(u_end - u_start))

            # Only accept full/near-full circles
            if arc_angle_deg >= min_circular_arc:
                valid_circular_edge = True
                break

        if not valid_circular_edge:
            continue

        # Passed all filters
        center = (
            round(location.X(), 3),
            round(location.Y(), 3),
            round(location.Z(), 3)
        )
        dir_vector = axis.Direction()
        direction = (
            round(dir_vector.X(), 3),
            round(dir_vector.Y(), 3),
            round(dir_vector.Z(), 3)
        )

        hole_info = {
            "Center": center,
            "Direction": direction,
            "Radius": round(radius, 3),
            "Diameter": round(diameter, 3)
        }
        hole_data.append(hole_info)

    return hole_data



In [3]:
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeSphere
from OCC.Core.gp import gp_Pnt
from OCC.Display.SimpleGui import init_display
from OCC.Core.ShapeAnalysis import ShapeAnalysis_Surface
from OCC.Core.GeomAPI import GeomAPI_ProjectPointOnSurf
from OCC.Core.BRep import BRep_Tool
from OCC.Core.BRepAdaptor import BRepAdaptor_Surface

def display_step(sample_shape, df_holes):
    # Start display
    display, start_display, _, _ = init_display()
    
    # Show the target shape
    display.DisplayShape(sample_shape, update=False)
    
    # Add red spheres at detected hole centers
    for hole in df_holes.to_dict(orient="records"):
        center = gp_Pnt(*hole["Center"])
        radius = min(2.0, hole["Radius"])  # Avoid very large spheres
        sphere = BRepPrimAPI_MakeSphere(center, radius).Shape()
        display.DisplayShape(sphere, color="RED", update=False)
        
        # Find the face where the hole is located and get the normal direction
        hole_face = find_face_for_hole(sample_shape, center)
        
        if hole_face is None:
            print(f"Warning: No face found for hole at center {hole['Center']}")
            continue  # Skip this hole if no valid face is found
        
        normal = get_face_normal(hole_face)  # Get the normal vector of the face
        
        # Adjust sphere placement based on the normal of the face
        hole_dir = gp_Vec(*hole["Direction"])
        if hole_dir.Angle(normal) < math.pi / 2:  # If the hole is aligned with the face's normal
            sphere.Translate(normal.Reversed() * 2)  # Adjust the position by 2 units along the normal
        display.DisplayShape(sphere, update=False)
    
    display.FitAll()
    start_display()


def find_face_for_hole(shape, hole_center):
    # Function to find the face where the hole is located (based on proximity to hole center)
    explorer = TopologyExplorer(shape)
    for face in explorer.faces():
        if is_point_on_face(face, hole_center):
            return face
    
    # Debug: Print to see what faces are explored
    print(f"No face found for hole at center: {hole_center}")
    return None

def is_point_on_face(face, point, tolerance=1e-3):
    # Get the geometric surface from the face
    surf = BRep_Tool.Surface(face)
    
    # Project the point onto the surface
    projector = GeomAPI_ProjectPointOnSurf(point, surf)
    
    if projector.NbPoints() == 0:
        return False  # Cannot project
    
    # Get the UV coordinates of the projection
    u, v = projector.LowerDistanceParameters()
    
    # Use BRepAdaptor_Surface to get the parameter bounds
    surf_adaptor = BRepAdaptor_Surface(face)
    
    # Check if (u, v) is within the bounds of the surface
    u_min, u_max = surf_adaptor.FirstUParameter(), surf_adaptor.LastUParameter()
    v_min, v_max = surf_adaptor.FirstVParameter(), surf_adaptor.LastVParameter()
    
    # Debug: Print UV and bounds
    # print(f"Projected UV: ({u}, {v}) | Bounds: U({u_min}, {u_max}), V({v_min}, {v_max})")
    
    # Check if the projected (u, v) is within bounds
    if not (u_min <= u <= u_max and v_min <= v <= v_max):
        return False
    
    # Check if the distance from the point to the projection is less than tolerance
    return surf_adaptor.Value(u, v).Distance(point) < tolerance



def get_face_normal(face):
    # Get the normal direction of the face
    surf_adaptor = BRepAdaptor_Surface()
    surf_adaptor.Initialize(face)
    u_min, u_max = surf_adaptor.FirstUParameter(), surf_adaptor.LastUParameter()
    v_min, v_max = surf_adaptor.FirstVParameter(), surf_adaptor.LastVParameter()
    u_mid, v_mid = (u_min + u_max) / 2, (v_min + v_max) / 2

    props = BRepLProp_SLProps(surf_adaptor, u_mid, v_mid, 1, 1e-6)
    normal = props.Normal()
    
    return gp_Vec(normal.X(), normal.Y(), normal.Z())





In [4]:
from OCC.Core.STEPControl import STEPControl_Reader
from OCC.Core.TopoDS import TopoDS_Shape
from OCC.Core.TopAbs import TopAbs_SOLID
from OCC.Extend.TopologyUtils import TopologyExplorer
from OCC.Core.BRepBndLib import brepbndlib
from OCC.Core.Bnd import Bnd_Box
from OCC.Display.SimpleGui import init_display
from OCC.Core.Quantity import Quantity_Color, Quantity_TOC_RGB
from OCC.Core.BRepGProp import brepgprop
from OCC.Core.GProp import GProp_GProps
from OCC.Core.gp import gp_Pnt
from OCC.Core.TCollection import TCollection_ExtendedString
from OCC.Core.BRepAdaptor import BRepAdaptor_Surface, BRepAdaptor_Curve
from collections import defaultdict
from IPython.display import display

from datetime import datetime
import random
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.precision', 3)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.colheader_justify', 'center')

from OCC.Core.BRepGProp import brepgprop_VolumeProperties, brepgprop_SurfaceProperties, brepgprop_LinearProperties

def extract_metadata(shape, label=""):
    metadata = {"Label": label}

    # Bounding Box
    bbox = Bnd_Box()
    brepbndlib.Add(shape, bbox)
    xmin, ymin, zmin, xmax, ymax, zmax = bbox.Get()
    metadata["BoundingBox_Min"] = (round(xmin, 2), round(ymin, 2), round(zmin, 2))
    metadata["BoundingBox_Max"] = (round(xmax, 2), round(ymax, 2), round(zmax, 2))
    metadata["Dimensions"] = (round(xmax - xmin, 2), round(ymax - ymin, 2), round(zmax - zmin, 2))

    # Volume Properties
    volume_props = GProp_GProps()
    brepgprop.VolumeProperties(shape, volume_props)
    metadata["Volume"] = round(volume_props.Mass(), 2)

    # Center of Gravity
    cog = volume_props.CentreOfMass()
    metadata["CenterOfGravity"] = (round(cog.X(), 2), round(cog.Y(), 2), round(cog.Z(), 2))

    # Surface Properties
    surface_props = GProp_GProps()
    brepgprop.SurfaceProperties(shape, surface_props)
    metadata["SurfaceArea"] = round(surface_props.Mass(), 2)

    # Linear Properties (optional - useful for wires/edges)
    linear_props = GProp_GProps()
    brepgprop.LinearProperties(shape, linear_props)
    metadata["Perimeter"] = round(linear_props.Mass(), 2)

    # Topological Information
    explorer = TopologyExplorer(shape)
    metadata["NumberOfFaces"] = sum(1 for _ in explorer.faces())
    metadata["NumberOfEdges"] = sum(1 for _ in explorer.edges())
    metadata["NumberOfVertices"] = sum(1 for _ in explorer.vertices())

    return metadata


def random_color():
    return Quantity_Color(random.random(), random.random(), random.random(), Quantity_TOC_RGB)

def load_step_file(file_path):
    reader = STEPControl_Reader()
    status = reader.ReadFile(file_path)
    if status != 1:
        raise Exception("Failed to read STEP file.")
    
    reader.TransferRoots()
    num_shapes = reader.NbShapes()
    return [reader.Shape(i) for i in range(1, num_shapes + 1)]

def get_all_solids(shapes):
    solids = []
    for shape in shapes:
        explorer = TopologyExplorer(shape)
        for solid in explorer.solids():
            solids.append(solid)
    return solids

def show_shapes(shapes):
    display, start_display, add_menu, add_function_to_menu = init_display()
    for shape in shapes:
        display.DisplayShape(shape, update=True)
    start_display()

def find_duplicate_geometries(metadata_list, tolerance=1e-3):
    """
    Find duplicates in metadata_list based on Volume, Dimensions, NumberOfFaces.
    Args:
        metadata_list: List of dicts containing shape metadata.
        tolerance: Allowable tolerance for volume and dimensions matching.
    Returns:
        DataFrame showing groups of duplicates.
    """
    # Helper to round numbers for fuzzy matching
    def rounded(val):
        if isinstance(val, (tuple, list)):
            return tuple(round(x, 3) for x in val)
        return round(val, 3)

    groups = defaultdict(list)

    for item in metadata_list:
        # Create a simple "signature" based on key properties
        signature = (
            rounded(item.get('Volume', 0)),
            rounded(item.get('Dimensions', (0,0,0))),
            item.get('NumberOfFaces', 0)
        )
        groups[signature].append(item["Label"])

    # Now filter to only groups with more than 1 item
    duplicate_groups = {k: v for k, v in groups.items() if len(v) > 1}

    # Convert to DataFrame for nice output
    data = []
    for group_idx, labels in enumerate(duplicate_groups.values(), 1):
        for label in labels:
            data.append({"DuplicateGroup": group_idx, "BodyLabel": label})

    df_duplicates = pd.DataFrame(data)
    return df_duplicates


# === USAGE IN JUPYTER ===

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)
file_path = "C25001-1-0101.step"  # <-- replace with your path

# Load STEP file and extract all shapes
top_shapes = load_step_file(file_path)
solids = get_all_solids(top_shapes)

# Limit to first few solids (for visualization and metadata)
sample = solids [:5]                                                    # You can change this to the number you want

# Optional labels (e.g. "Body 1", "Body 2", ...)
labels = [f"Body {i+1}" for i in range(len(sample))]

# Display shapes with color and labels
# show_shapes_with_labels(sample, labels)

# Extract metadata for the displayed solids
metadata_list = []
for shape, label in zip(sample, labels):
    data = extract_metadata(shape, label)
    metadata_list.append(data)

# Convert the metadata list to a Pandas DataFrame
df_metadata = pd.DataFrame(metadata_list)

#duplicate geometry search
df_duplicates = find_duplicate_geometries(metadata_list)

# NC1 Creation
sample_shape = solids[25]  # Replace with the specific solid you want to export

# Extract points from the solid
points = extract_points_from_shape(sample_shape)

# Extract holes from shape
df_holes = pd.DataFrame(detect_holes(sample_shape))


print("Duplicates Search results")
styled_duplicates = df_duplicates.style.set_properties(
    **{
        'background-color': '#f9f9f9',
        'border-color': 'black',
        'border-style': 'solid',
        'border-width': '1px',
        'color': 'black',
        'font-size': '14px',
        'text-align': 'center'
    }
    ).set_table_styles(
        [{
            'selector': 'thead th',
            'props': [('background-color', '#404040'), 
                      ('color', 'white'), 
                      ('font-size', '16px')]
        }]
    ).hide(axis='index')  # Hide the row indices if you want

display(styled_duplicates)

# Display the full metadata table
print("Item Search Results")

styled_metadata = df_metadata.style.set_properties(
        **{
            'background-color': '#f9f9f9',
            'border-color': 'black',
            'border-style': 'solid',
            'border-width': '1px',
            'color': 'black',
            'font-size': '14px',
            'text-align': 'center'
        }
    ).set_table_styles(
        [{
            'selector': 'thead th',
            'props': [('background-color', '#404040'),
                      ('color', 'white'),
                      ('font-size', '16px')]
        }]
    ).hide(axis='index')

display(styled_metadata)

print(f"Holes analysis of {sample_shape}")

styled_holes = df_holes.style.set_properties(
        **{
            'background-color': '#f9f9f9',
            'border-color': 'black',
            'border-style': 'solid',
            'border-width': '1px',
            'color': 'black',
            'font-size': '14px',
            'text-align': 'center'
        }
    ).set_table_styles(
        [{
            'selector': 'thead th',
            'props': [('background-color', '#404040'),
                      ('color', 'white'),
                      ('font-size', '16px')]
        }]
    ).hide(axis='index')

display(styled_holes)

display_step(sample_shape, df_holes)

# Write the points to an NC1 file
write_nc1_file(points, "output.nc1")

print("NC1 file generated!")


Current Time = 21:34:19
Duplicates Search results


DuplicateGroup,BodyLabel
1,Body 4
1,Body 5


Item Search Results


Label,BoundingBox_Min,BoundingBox_Max,Dimensions,Volume,CenterOfGravity,SurfaceArea,Perimeter,NumberOfFaces,NumberOfEdges,NumberOfVertices
Body 1,"(5729.93, 29.71, 6065.0)","(6066.05, 479.71, 6077.0)","(336.12, 450.0, 12.0)",1208280.190000,"(5943.87, 286.8, 6071.0)",222548.050000,7464.010000,19,51,34
Body 2,"(5994.9, -675.0, 5994.8)","(6147.1, -525.0, 6147.2)","(152.2, 150.0, 152.4)",438681.240000,"(6071.0, -600.0, 6071.0)",139191.910000,8355.810000,18,48,32
Body 3,"(5771.0, -525.0, 5771.0)","(6371.0, -485.0, 6371.0)","(600.0, 40.0, 600.0)",14327617.710000,"(6071.0, -505.0, 6071.0)",824444.600000,11766.370000,14,36,24
Body 4,"(6075.95, 16480.0, 5932.45)","(6220.95, 16500.0, 6209.55)","(145.0, 20.0, 277.1)",791090.000000,"(6149.46, 16490.0, 6071.0)",95407.210000,3499.640000,8,18,12
Body 5,"(6075.95, 15463.9, 5932.45)","(6220.95, 15483.9, 6209.55)","(145.0, 20.0, 277.1)",791090.000000,"(6149.46, 15473.9, 6071.0)",95407.210000,3499.640000,8,18,12


Holes analysis of <class 'TopoDS_Solid'>


Center,Direction,Radius,Diameter
"(5729.924, 16655.0, 6136.0)","(1.0, 0.0, 0.0)",11.000000,22.000000
"(5729.924, 16655.0, 6006.0)","(1.0, 0.0, 0.0)",11.000000,22.000000
"(5729.924, 15935.0, 6006.0)","(1.0, 0.0, 0.0)",13.000000,26.000000
"(5729.924, 15935.0, 6136.0)","(1.0, 0.0, 0.0)",13.000000,26.000000
"(5729.924, 16045.0, 6006.0)","(1.0, 0.0, 0.0)",13.000000,26.000000
"(5729.924, 16045.0, 6136.0)","(1.0, 0.0, 0.0)",13.000000,26.000000
"(5729.924, 16155.0, 6006.0)","(1.0, 0.0, 0.0)",13.000000,26.000000
"(5729.924, 15355.0, 6006.0)","(1.0, 0.0, 0.0)",13.000000,26.000000
"(5729.924, 15245.0, 6006.0)","(1.0, 0.0, 0.0)",13.000000,26.000000
"(5729.924, 15245.0, 6136.0)","(1.0, 0.0, 0.0)",13.000000,26.000000


pyside6 backend - Qt version 6.8.3
Projected UV: (3.141592653589765, -0.0003005866765306564) | Bounds: U(3.141592653589793, 6.283185307179586), V(336.12569941332146, 346.0256994133215)
Projected UV: (1.7494273333088484, -0.0003005866765306564) | Bounds: U(3.141592653589793, 6.283185307179586), V(336.12569941332146, 346.0256994133215)
Projected UV: (1.5707963267948992, -0.0003005866765306564) | Bounds: U(3.141592653589793, 6.283185307179586), V(336.12569941332146, 346.0256994133215)
Projected UV: (1.7807698396570635, -0.0003005866765306564) | Bounds: U(3.141592653589793, 6.283185307179586), V(336.12569941332146, 346.0256994133215)
Projected UV: (1.5707963267948997, -0.0003005866765306564) | Bounds: U(3.141592653589793, 6.283185307179586), V(336.12569941332146, 346.0256994133215)
Projected UV: (1.825164385348167, -0.0003005866765306564) | Bounds: U(3.141592653589793, 6.283185307179586), V(336.12569941332146, 346.0256994133215)
Projected UV: (1.67046497928606, -0.0003005866765306564) | Bo

Export Holes to JSON File

In [56]:
import json

holes = detect_holes(sample_shape)
with open("holes.json", "w") as f:
    json.dump(holes, f)


In [5]:
import matplotlib.pyplot as plt

def is_point_on_face(face, point, tolerance=1e-1):
    # Get the geometric surface from the face
    surf = BRep_Tool.Surface(face)
    
    # Project the point onto the surface
    projector = GeomAPI_ProjectPointOnSurf(point, surf)
    
    if projector.NbPoints() == 0:
        print(f"Cannot project point {point} onto the surface.")
        return False  # Cannot project
    
    # Get the UV coordinates of the projection
    u, v = projector.LowerDistanceParameters()
    print(f"Point {point} projected onto (u, v): ({u}, {v})")
    
    # Now check if (u,v) lies within the bounds of the face
    sas = ShapeAnalysis_Surface(surf)
    u_min, u_max = sas.FirstUParameter(), sas.LastUParameter()
    v_min, v_max = sas.FirstVParameter(), sas.LastVParameter()

    print(f"UV bounds for the surface: u: ({u_min}, {u_max}), v: ({v_min}, {v_max})")
    
    # Check if (u, v) lies within the bounds of the surface
    if u < u_min or u > u_max or v < v_min or v > v_max:
        return False  # (u,v) is out of bounds

    return sas.Value(u, v).Distance(point) < tolerance


def find_face_for_hole(shape, hole_center):
    # Function to find the face where the hole is located (based on proximity to hole center)
    explorer = TopologyExplorer(shape)
    for face in explorer.faces():
        if is_point_on_face(face, hole_center):
            return face
    return None

def display_points_with_status(points, status):
    # Plot points with different colors based on the status (found face or not)
    found_points = [point for point, s in zip(points, status) if s]
    not_found_points = [point for point, s in zip(points, status) if not s]
    
    # Plot the found points in blue and the not found points in red
    plt.scatter(*zip(*found_points), color='blue', label='Face Found')
    plt.scatter(*zip(*not_found_points), color='red', label='No Face Found')
    
    # Add labels and a legend
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.legend()
    plt.show()

# Example usage:
points = [(5729.924, 16655.0, 6136.0), (5000.0, 15000.0, 6000.0)]  # Sample points
status = []  # Keep track of whether each point is successfully projected

# Check each point and determine if it finds a face
for point in points:
    hole_center = gp_Pnt(*point)  # Convert to gp_Pnt object
    hole_face = find_face_for_hole(sample_shape, hole_center)
    
    # If a face is found, mark the point as True, otherwise False
    status.append(hole_face is not None)

# Display the points with different colors based on the status
display_points_with_status(points, status)


Cannot project point <class 'gp_Pnt'> onto the surface.
Point <class 'gp_Pnt'> projected onto (u, v): (3.141592653589765, -0.0003005866765306564)


AttributeError: 'ShapeAnalysis_Surface' object has no attribute 'FirstUParameter'